In [1]:
import numpy as np
import torch
from torch import nn
import os
from tqdm.notebook import tqdm

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, self.hidden_size).to(x.device) 
        c0 = torch.zeros(self.num_layers * 2, self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

# Prepare data
X = []
y = []
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X.append(file['features'])
        y.append(file['labels'])
N = len(y)

X_test = X[0:N//4]
y_test = y[0:N//4]

X_train = X[N//4:N]
y_train = y[N//4:N]
n = len(y_train)
# print(torch.cuda.is_available())

def train(inputs, outputs):
    # Parameters
    hidden_size = 256
    input_size = 105
    output_size = 7
    num_layers = 2
    num_epochs = 100
    learning_rate = 0.01
    batch_size = 32

    # Instantiate the model
    model = LSTMModel(input_size, hidden_size, num_layers, output_size)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device='cuda')
    criterion = criterion.to(device='cuda')

    n = len(inputs)
    # Training loop
    for epoch in range(num_epochs):
        total_loss=0
        slices = np.random.randint(0,n-1,(batch_size,))
        for j in tqdm(range(batch_size)):
            i = slices[j]
            input = torch.tensor(inputs[i], dtype=torch.float32).to(device='cuda')
            output = torch.tensor(outputs[i], dtype=torch.long).to(device='cuda')
            output_pred = model(input)
            optimizer.zero_grad()
            loss = criterion(output_pred, output)
            total_loss += loss
            loss.backward()
            optimizer.step()

        # if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item()}')
    return model


In [2]:
model = train(X_train, y_train)

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1/100, Loss: 26.21760368347168


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 2/100, Loss: 20.542049407958984


  0%|          | 0/32 [00:00<?, ?it/s]

...

Epoch 98/100, Loss: 12.164592742919922


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 99/100, Loss: 11.329961776733398


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 100/100, Loss: 10.103872299194336


In [3]:

import scoring
# Parameters
hidden_size = 256
input_size = 105
output_size = 7
num_layers = 2
num_epochs = 50
learning_rate = 0.01
batch_size = 32
    
def predict(model, inputs):
    y = []
    for input in tqdm(inputs):
        y += predict_one(model, input)
        # print(y)
    return y

def predict_one(model, input):
    output_pred = model(torch.tensor(input, dtype=torch.float32).to(device='cuda'))
    N = len(input)
    y = []
    for i in range(N):
        mx = output_pred[i][0]
        p = 0
        for j in range(1,output_pred.shape[1]):
            if output_pred[i][j] > mx:
                mx = output_pred[i][j]
                p = j
        y.append(p)
    return y

y_train_pred = predict(model, X_train)
y_test_pred = predict(model, X_test)

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

In [4]:
print('train score:')
y_true = []
for y in y_train:
    y_true += y.tolist()
scoring.score(y_train_pred, y_true)
print('test score:')
y_true = []
for y in y_test:
    y_true += y.tolist()
scoring.score(y_test_pred, y_true)

train score:
Class 0: Precision: 0.955, Recall: 0.981
Class 1: Precision: 0.396, Recall: 0.365
Class 2: Precision: 0.792, Recall: 0.860
Class 3: Precision: 0.617, Recall: 0.021
Class 4: Precision: 0.650, Recall: 0.161
Class 5: Precision: 0.847, Recall: 0.625
Class 6: Precision: 0.981, Recall: 0.226
mf1 score: 0.572
acc_score: 0.897
kappa_score: 1.037
test score:
Class 0: Precision: 0.928, Recall: 0.985
Class 1: Precision: 0.110, Recall: 0.242
Class 2: Precision: 0.694, Recall: 0.729
Class 3: Precision: 0.364, Recall: 0.002
Class 4: Precision: 0.311, Recall: 0.012
Class 5: Precision: 0.751, Recall: 0.485
Class 6: Precision: 0.000, Recall: 0.000
mf1 score: 0.395
acc_score: 0.849
kappa_score: 0.708
